In [27]:
#import the liberary
import requests
from time import sleep 
import re
import pandas as pd
from bs4 import BeautifulSoup

In [28]:
#import the liberary
import requests as rq
from bs4 import BeautifulSoup as bs
from urllib.parse import urlencode

In [29]:
# define header to access google scholar website
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'}

In [30]:
# this function for the getting inforamtion of the web page
def get_paperinfo(paper_url):
    response=requests.get(url,headers=headers)

  # check successful response
    if response.status_code != 200:
        print('Status code:', response.status_code)
        raise Exception('Failed to fetch web page ')

       #parse using beautiful soup
    paper_doc = BeautifulSoup(response.text,'html.parser')
 
    return paper_doc

In [31]:
# this function for the extracting information of the tags
def get_tags(doc):
    paper_tag = doc.select('[data-lid]')
    
    #cite_tag = doc.select('[title=Cite] + a')
    
    #cite_tag = doc.find_all('a', text=re.compile(r'\d+'))
    #cite_tag = f"https://scholar.google.com/scholar?q={doc}"
    #cite_tag = doc.find_all('a.gs_or_cit')
    #cite_tag = doc.find_all('a', text=re.compile(r'\d+'))
    
    cite_tag = doc.find_all('a', text=re.compile(r'^Cited by \d+'))

    #cite_tag = doc.find_all('#gs_res_ccl_mid .gs_nph+ a')['href']
    
    link_tag = doc.find_all('h3',{"class" : "gs_rt"})
    author_tag = doc.find_all("div", {"class": "gs_a"})

    return paper_tag,cite_tag,link_tag,author_tag

In [32]:
# it will return the title of the paper
def get_papertitle(paper_tag):
  
    paper_names = []
  
    for tag in paper_tag:
        paper_names.append(tag.select('h3')[0].get_text())

    return paper_names

In [33]:
def get_citecount(cite_tag):
    citation_count = []
    for tag in cite_tag:
        cite_text = tag.text.strip()
        tmp = re.search(r'\d+', cite_text)
        if tmp is None:
            citation_count.append(0)
        else:
            citation_count.append(int(tmp.group()))
# Check if there are citation tags with zero count
    if len(cite_tag) == 0:
            citation_count.append(0) 
    return citation_count

In [34]:
# def get_citecount(cite_tag):
#     """Finds all the citations in the document."""
#    # cite_tag = doc.find_all('a', text=re.compile(r'\d+'))

#     citations = []
#     for cite_tag in cite_tag:
#         # Check if the citation contains "Cited by"
#         if "Cited by" in cite_tag.text:
#             citations.append((cite_tag.text, cite_tag['href']))
#         else:
#             # If the citation does not contain "Cited by", assume that the
#             # citation is just the number of citations.
#             citations.append((cite_tag.text, 0))

#     return citations

In [35]:
# function for the getting link information
def get_link(link_tag):
    links = []

    for i in range(len(link_tag)) :
        links.append(link_tag[i].a['href']) 

    return links 

In [36]:
# function for the getting autho , year and publication information
def get_author_year_publi_info(authors_tag):
    years = []
    publication = []
    authors = []
    for i in range(len(authors_tag)):
        authortag_text = (authors_tag[i].text).split()
        year = int(re.search(r'\d+', authors_tag[i].text).group())
        years.append(year)
        publication.append(authortag_text[-1])
        author = authortag_text[0] + ' ' + re.sub(',','', authortag_text[1])
        authors.append(author)
    return years , publication, authors

In [37]:
#a = {'Links' : lines ,'Titles' : titles , 'Singers': finalsingers , 'Albums':finalalbums , 'Years' : years}
#df = pd.DataFrame.from_dict(a, orient='index')
#df = df.transpose()

In [38]:
# creating final repository
paper_repos_dict = {
                    'Paper Title' : [],
                    'Year' : [],
                    'Author' : [],
                    'Citation' : [],
                    'Publication' : [],
                    'Url of paper' : [] }

# adding information in repository
def add_in_paper_repo(papername,year,author,cite, publication,link):
    paper_repos_dict['Paper Title'].extend(papername)
    paper_repos_dict['Year'].extend(year)
    paper_repos_dict['Author'].extend(author)
    paper_repos_dict['Citation'].extend(cite)
    paper_repos_dict['Publication'].extend(publication)
    paper_repos_dict['Url of paper'].extend(link)

    return pd.DataFrame.from_dict(paper_repos_dict)

In [20]:
for i in range (0,110,10):
    url = "https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=locational+marginal+price+LMP+&oq=".format(i)

  # function for the get content of each page
    doc = get_paperinfo(url)
    paper_tag,cite_tag,link_tag,author_tag = get_tags(doc)
  
  # paper title from each page
    papername = get_papertitle(paper_tag)

  # year , author , publication of the paper
    year , publication , author = get_author_year_publi_info(author_tag)

  # cite count of the paper 
    cite = get_citecount(cite_tag)

  # url of the paper
    link = get_link(link_tag)

  # add in paper repo dict
    final = add_in_paper_repo(papername, year, publication,cite, author, link)
    #final = final.transpose()
  
  # use sleep to avoid status code 429
    sleep(30)

In [21]:
final[:10]

,Paper Title,Year,Author,Citation,Publication,Url of paper
0,[PDF][PDF] Locational Marginal Price (LMP) Ass...,2010,academia.edu,203,H Zeynal,https://www.academia.edu/download/48839157/Loc...
1,Impact of data quality on real-time locational...,2013,ieeexplore.ieee.org,128,L Jia,https://ieeexplore.ieee.org/abstract/document/...
2,Design and operation of the locational margina...,2010,IET,158,E Litvinov,https://digital-library.theiet.org/content/jou...
3,Continuous locational marginal pricing (CLMP),2007,ieeexplore.ieee.org,128,F Li,https://ieeexplore.ieee.org/abstract/document/...
4,A novel policy for locational marginal price c...,2012,ieeexplore.ieee.org,155,K Shaloudegi,https://ieeexplore.ieee.org/abstract/document/...
5,Locational marginal pricing basics for restruc...,2009,ieeexplore.ieee.org,85,H Liu,https://ieeexplore.ieee.org/abstract/document/...
6,Assessment of transmission congestion cost and...,2004,ieeexplore.ieee.org,136,G Hamoud,https://ieeexplore.ieee.org/abstract/document/...
7,Introducing uncertainty components in location...,2019,ieeexplore.ieee.org,81,X Fang,https://ieeexplore.ieee.org/abstract/document/...
8,[HTML][HTML] Deregulated model and locational ...,2007,Elsevier,52,YR Sood,https://www.sciencedirect.com/science/article/...
9,Some observations on market clearing price and...,2005,ieeexplore.ieee.org,46,Z Li,https://ieeexplore.ieee.org/abstract/document/...


In [22]:
#Verification for the len
print(len(papername), len(year), len(publication), len(cite), len(author),  len( link))


10 10 10 10 10 10


In [23]:
cite

[203, 128, 158, 128, 155, 85, 136, 81, 52, 46]

In [25]:
## Save as CSV file
final.to_csv('Locational_marginal_price.csv', sep=',', index=False,header=True)